## Anatomy of a Car.

You design a new car by specifying a series of sizes. They are grouped into the following categories:

1. the frame length (total length of the car);
2. the shape of the upper profile;
3. the shape of the lower profile;
4. the first wheel;
5. the second wheel.

See the picture below for reference:

<img src="https://sales.bio.unipd.it/assets/carsim/car_model.png" width="600px">

### 1. Frame Length

This is a single floating point number representing the entire length of the car, from the front to the rear. The allowed range is comprised between 2.0 and 6.0 meters.

### 2. Upper Shape

The upper profile of the car is described by 5 floating point numbers. Each of them describes the height of the frame at equi‑spaced points along the car, from left (back) to right (front).

Example: the array [0.5, 0.7, 1.3, 0.8, 0.5] would correspond to the picture above.

### 3. Lower Shape

The lower profile of the car is described by 5 floating point numbers. Each of them describes the height of the frame at equi‑spaced points along the car, from left (back) to right (front).

Example: the array [0.5, 0.7, 0.4, 0.6, 0.5] would correspond to the picture above.

### Shape Constraints

For manufacturing purposes, the car cannot be either too high or too thin. In numbers, this mean that the **total** height (lower shape + upper shape at any of the 5 provided points) must be comprised between 0.5 and 5.0.  

Example: the sum of the lower and the upper shapes shown before is

`[0.5, 0.7, 1.3, 0.8, 0.5] + [0.5, 0.7, 0.4, 0.6, 0.5] = [1.0, 1.4, 1.7, 1.4, 1.0]`

This car respects the manufacturing constraints.

### 4. Left Wheel

The left wheel is defined by two numbers.

1. its position along the car frame. A real number between 0.0 (wheel at the extreme left, back of the car) and 1.0 (wheel at the extreme right, front of the car);
2. its radius (between 0 and 2 meters).

Example: the left wheel of the picture above is defined as [0.1, 0.5].

### 5. Right Wheel

The right wheel is defined by two numbers.

1. its position along the car frame. A real number between 0.0 (wheel at the extreme left, back of the car) and 1.0 (wheel at the extreme right, front of the car);
2. its radius (between 0 and 2 meters).

Example: the right wheel of the picture above is defined as [0.9, 0.8].

# Simulator

To make the car run along a track, we will use a simulator provided by the test driving team.

You will need to import a few libraries.

In [3]:
import carsim
import numpy as np

Simulator started.

Now open the webpage at this address in a separate browser window: http://localhost:41174/


Do as the text above tells you. Open that URL in a **separate** browser window (right click on the link and select "Open in a new window").

From this point on you have access to three useful functions.

### carsim.simulate

This function takes the following arguments: `frame_length, upper_shape, lower_shape, left_wheel, right_wheel`. That is, the spec of a car.

It then build the vehicle and brings it to the test track. You can follow the test drive in the other browser window (titled "Car Simulator Dashboard").

### carsim.race

Takes a list of car blueprints. Each car is represented as a tuple of the form:

```(frame_length, upper_shape, lower_shape, left_wheel, right_wheel)```

The function will accept any number of cars between 1 and 100 (included).

It then starts a race among all the cars, measuring how well each performed on the track.

The return value `res` is a N by 2 matrix (where N is the number of cars).
* the first column contains the fraction of the track that was covered by the car (0.0 meaning the car didn't even start; 1.0 meaning that the car reached the finish line);
* the second column measures how many seconds it took to the car to finish its race.

Example: res[2, 1] is how long the race of the third car lasted.

<div class="alert alert-warning" style="margin-top:1em">
The race <strong>won't</strong> be shown in the Simulator Dashboard.
</div>

### carsim.new_track

If you don't like the race track, you can change the location with this function. The new track will be used for the following simulations and races.

## Your Turn

It is now time to design the best possible car for offroad racing.

Develop a genetic algorithm for testing out different random designs and to pick the best performing car.

You may find the following functions useful:

* *np.random.rand*: generate a random integer in the interval [0, 1).

In [4]:
np.random.rand()

0.799677618084832

If you specify a length, it will return that many random numbers.

In [5]:
np.random.rand(10)

array([0.6487639 , 0.17209866, 0.80792937, 0.30513797, 0.49728114,
       0.22909067, 0.49631888, 0.42687259, 0.5103542 , 0.19961913])

* np.random.rand(): generate a random number *normally distributed* with mean 0 and variance 1.

In [6]:
np.random.randn(5)

array([ 3.07148496,  1.99760657, -0.0912822 ,  0.93359913,  0.23250073])

* *np.min* and *np.max*: compute the minimum / maximum of a sequence of numbers.

In [7]:
np.min(np.arange(10))

np.int64(0)

In [8]:
np.max([1, 3, 2, 0])

np.int64(3)

* *np.clip*: clip values to a given range.

    Given a value `v`, a lower bound `lower` and an upper bound `upper`, makes sure that `lower <= v <= upper`. 

In [9]:
np.clip(3,  # value
        0,  # lower bound
        4)  # upper bound

np.int64(3)

In [10]:
np.clip(-1, 0, 4)

np.int64(0)

In [11]:
np.clip(5, 0, 4)

np.int64(4)

In [12]:
values = np.arange(10)
values

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [13]:
np.clip(values, 3, 5)

array([3, 3, 3, 3, 4, 5, 5, 5, 5, 5])

By default, `np.clip` returns a new array. It is possible to overwrite the original (input) array with the following syntax.

In [14]:
np.clip(values, 3, 5, out=values)
values

array([3, 3, 3, 3, 4, 5, 5, 5, 5, 5])

* *np.argsort*: return the positions of the elements of an array from the smallest to the largest. 

In [15]:
numbers = np.array([3., 1., 2])
np.argsort(numbers)

array([1, 2, 0])

This means that `numbers[1]` has the smallest value; then comes `numbers[2]`; and finally `numbers[0]`, which has also the largest value.

## Implementation

### Step 1

To develop our genetic algorithm for cars, we first need to decide how to represent their genome. Remember that the  blueprint of a car is defined by:
* length;
* upper shape;
* lower shape;
* left wheel position / size;
* right wheel position / size.

For instance:

## Explanation Atefe: 

In a genetic algorithm, we treat each solution (a car, here) as a "genome" — a 1D array of values that represents everything about the car design.

In [16]:
blueprint = [
    4.,  # length
    np.array([.8, 1., 1., .5, .3]),  # upper profile
    np.array([.3, .3, .3, .3, .3]),  # lower profile
    np.array([.1, .5]),  # left wheel (position, size)
    np.array([.9, .5]),  # right wheel (position, size)
]

This is not a genome, though. A genome is simply a flat sequence of numbers, just like an array. Example:

In [17]:
genome = np.array([4., 1., 1., 1., 1., 1., .5, .5, .5, .5, .5, .1, 1., .9, 1.])

Develop a pair of functions to:
1. convert a blueprint like the above into the corresponding genome;
1. convert the genome back into a blueprint.

In [18]:
def car_genome(car):
    """
    Convert car blueprint to genome (1D np.array).
    """
    frame_length = np.array([car[0]])
    upper = car[1]
    lower = car[2]
    left = car[3]
    right = car[4]
    
    genome = np.concatenate([frame_length, upper, lower, left, right])
    return genome

In [19]:
car_genome(blueprint)

array([4. , 0.8, 1. , 1. , 0.5, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.1, 0.5,
       0.9, 0.5])

In [20]:
def car_from_genome(genome):
    """
    Convert genome (1D np.array) back to blueprint format.
    """
    frame_length = genome[0]
    upper = genome[1:6]
    lower = genome[6:11]
    left = genome[11:13]
    right = genome[13:15]
    
    blueprint = [
        frame_length,
        upper,
        lower,
        left,
        right
    ]
    return blueprint

In [21]:
car_from_genome(genome)

[np.float64(4.0),
 array([1., 1., 1., 1., 1.]),
 array([0.5, 0.5, 0.5, 0.5, 0.5]),
 array([0.1, 1. ]),
 array([0.9, 1. ])]

### Step 2

Initially, a genetic algorithm has no idea of which design would work the best for a given task. It creates blueprint totally at random.

Complete the function below, which should return a random car blueprint.

**Note**: not all random combinations of numbers are acceptable. For instance, the frame length must be comprised between 2 and 6 meters.

**Recap**

Frame length must be in [2.0, 6.0]

Upper shape: 5 floats

Lower shape: 5 floats

➤ Combined total height at each point (upper + lower) must be in [0.5, 5.0]

Wheel positions: floats in [0.0, 1.0]

Wheel radii: floats in [0.0, 2.0]

In [22]:
def random_car():
    frame_length = np.random.uniform(2.0, 6.0)
    
    # Generate valid upper/lower shapes
    while True:
        upper = np.random.uniform(0.0, 3.0, size=5)
        lower = np.random.uniform(0.0, 2.0, size=5)
        total = upper + lower
        if np.all((0.5 <= total) & (total <= 5.0)):
            break

    # Generate wheels, ensure left is left of right
    while True:
        left_pos = np.random.rand()
        right_pos = np.random.rand()
        if left_pos < right_pos:
            break

    left = np.array([left_pos, np.random.uniform(0.0, 2.0)])
    right = np.array([right_pos, np.random.uniform(0.0, 2.0)])

    return [frame_length, upper, lower, left, right]


Now generate a random car and simulate it.

**Note**: what should you write in place of the ellipses below? Can you use the variable `car1` directly? Why? Is there any way to make it work?

In [23]:
car1 = random_car()
carsim.simulate(*car1)

In [24]:
car1

[3.4907390908171023,
 array([1.76898273, 0.73887283, 0.81365558, 0.39929305, 2.67758998]),
 array([1.07603489, 0.67040588, 1.30043418, 1.57343767, 1.29537024]),
 array([0.15265237, 0.57387056]),
 array([0.62489535, 0.40855924])]

[4.3664917389344895,
 array([0.96386247, 0.24849133, 0.84172929, 0.30730235, 1.1393733 ]),
 array([0.13014457, 1.79371071, 1.22841169, 0.39929371, 0.59288133]),
 array([0.12555719, 1.23529497]),
 array([0.81229723, 1.21390012])]

### Step 3

We need to evaluate the fitness of our car, that is how good it was at completing the track.

Try the following:

In [25]:
carsim.race([car1])

array([[0.0767062, 5.93333  ]])

array([[ 1. , 17.2]])


Here we are making a (boring) race with a single car. The interesting thing is that the `race` function returned two useful numbers:
1. the fraction of the track that was completed by the car (0 is bad, 1 is good);
1. the race time (as long as the car completed the track, lower means faster and thus better).

If you make two cars race, you will get a pair of numbers for each car.

In [26]:
car2 = random_car()
carsim.race([car1, car2])

array([[0.0767062, 5.93333  ],
       [0.0630844, 5.26667  ]])

res = np.array([
    [0.85, 12.7],   # car1: 85% of track completed, finished in 12.7 seconds
    [1.00, 10.4]    # car2: 100% completed, faster time
])

### Step 4

Things are starting to take shape. We now have a way to generate random cars and function to evaluate their respective fitness.

The next logical step would be to generate an initial population of random cars (say 50) and then to identify the most primising ones.

Complete the following functions:

In [27]:
def initial_population(size):
    cars = []
    for _ in range(size):
        car = random_car()
        cars.append(car)
    return cars

In [28]:
# Example usage
population = initial_population(50)
print(f"Generated initial population of {len(population)} cars.")


Generated initial population of 50 cars.


### Step 5

If you make your car population race, you will find somewhat difficult to find the best cars. Your task would probably be easier if the score returned by `carsim.race` would actually consist in a single number. You could simply sort everything from the largest value to the smallest. Here, on the other hand, you got two values for each car (fraction of the track covered, elapsed time).

Leaving programming on the side for a moment, can you think of any reasonable approach to combine those two numbers? Would it make sense to simply sum the two of them? Is any of the two numbers redundant? Alternatively, is that the case only under certain conditions?

When you have found a solution to this problem, complete the function below. `combined_score` should take the output of `carsim.race` and return an array of single‑number scores, one for each car.

[completion_fraction, time_to_finish]

✅ Metric 1: Completion Fraction
Most important!

If a car didn't finish, the time is meaningless

A car that finished 100% is better than one that did 99%, no matter how fast

✅ Metric 2: Time (only relevant if car finished)
Among finished cars, lower time is better


if (completion < 1.0) -> use just the completion as the score.
If a car did finish -> subtract the time from a large constant (e.g., 100) to reward speed.

In [29]:
def combined_scores(scores):
    combined = []
    for completion, time in scores:
        if completion < 1.0:
            score = completion  # didn't finish: use only track fraction
        else:
            score = 1.0 + (100.0 - time)  # finished: bonus + faster = better
        combined.append(score)
    return np.array(combined) # 1-D array of score numbers

In [30]:
import numpy as np

# Sample race result for 3 cars
scores = np.array([
    [1.0, 12.5],   # finished
    [1.0, 14.8],   # finished
    [0.8, 17.2],   # not finished
])

print(combined_scores(scores))
# Output: something like [88.5, 86.2, 0.8]


[88.5 86.2  0.8]


Armed with this function, let's find out the best performing cars.

Implement `select_best`, a function taking:
1. a car population, as returned by `initial_population`;
2. scores: the output of `carsim.race` on those cars;
3. fraction: a number between 0 and 1.

`select_best` should rank all the cars according to their score and return only the best ones. The exact number is determined by fraction: if it's .3, the function should return 30% of the original population size, picking only the best individuals.

In [31]:
def select_best(population, scores, fraction):
    # Get 1D score array
    combined = combined_scores(scores)
    
    # Sort indices from best to worst
    sorted_indices = np.argsort(-combined)  # Negative for descending order
    
    # Compute how many to keep
    n = int(len(population) * fraction)
    
    # Select top n cars
    best = [population[i] for i in sorted_indices[:n]]
    
    return best

In [32]:
# Example usage
# Generate a population of 50 cars
population = initial_population(50)

# Run the race
race_results = carsim.race(population)

# Select the top 90% best cars
best_cars = select_best(population, race_results, 0.9)

print(f"Selected {len(best_cars)} best cars.")


Selected 45 best cars.


In [33]:
# for i, car in enumerate(best_cars):
#     print(f"Showing best car #{i + 1}")
#     carsim.simulate(*car)
    
#     # Optional: pause between cars
#     input("Press Enter to show the next car...")


### Step 6

Taking inspiration from biology, we have realized some sort of *selection* based on a fitness criterium.

This is only half of our path towards a genetic algorithm. Now we want *evolution*. We want to simulate the mating of two individuals producing an offspring which displays some of the traits of each parent.

Let's reach that goal in small steps. First of all implement the function `mate`. It should take as input two cars and it should generate a third one inheriting part of its genome from car 1 and part from car 2.

**Note**: remember that in *Step 1* you have written two useful functions, `car_genome` and `car_from_genome`.

In [34]:
def mate(car1, car2):
    # Convert cars to genomes
    genome1 = car_genome(car1)
    genome2 = car_genome(car2)

    # Choose a crossover point (not at the very ends)
    crossover_point = np.random.randint(1, len(genome1))

    # Combine genes
    child_genome = np.concatenate([
        genome1[:crossover_point],
        genome2[crossover_point:]
    ])

    # Convert genome back to car
    car3 = car_from_genome(child_genome)

    return car3


Now make a small improvement to the above implementation: add some random mutation into the mix.

Generate some (small) random values and add those to the genome of the offspring. Which random number should you use? What should the expected range of the random values be?

In [35]:
def mate(car1, car2, mutation_strength=0.05):
    # Convert cars to genomes
    genome1 = car_genome(car1)
    genome2 = car_genome(car2)

    # Crossover
    crossover_point = np.random.randint(1, len(genome1))
    child_genome = np.concatenate([
        genome1[:crossover_point],
        genome2[crossover_point:]
    ])

    # Mutation: small Gaussian noise
    mutation = mutation_strength * np.random.randn(len(child_genome))
    child_genome += mutation

    # Clip values to valid ranges
    child_genome[0] = np.clip(child_genome[0], 2.0, 6.0)         # frame length
    child_genome[1:6] = np.clip(child_genome[1:6], 0.0, None)    # upper
    child_genome[6:11] = np.clip(child_genome[6:11], 0.0, None)  # lower
    child_genome[11:13] = np.clip(child_genome[11:13], 0.0, 1.0) # wheel positions
    child_genome[12] = np.clip(child_genome[12], 0.0, 2.0)       # left radius
    child_genome[14] = np.clip(child_genome[14], 0.0, 2.0)       # right radius

    # Convert back to car
    child_car = car_from_genome(child_genome)

    return child_car

### Step 7

If you take a close look to the output of the `mate` function updated with mutation, you will notice that it sometimes produces an invalid result. For instance, it may generate a frame length which is longer than 6 meters.

Write a function `adjust_car` which:
1. takes as input a car;
2. if needed, alters its measures to make them compatible with the provided constraints;
3. returns the adjusted car.

For instance, in the above case (frame length > 6 meters) the `adjust_car` should reduce the frame length to 6 meters. All other measures may remain the same, as long as they are themeselves valid.

**Note**: it may be easier to build `adjust_car` from smaller functions. Consider: `adjust_shapes` (which fixes the upper and lower shapes) and `adjust_wheel` (which fixes one wheel at a time).

In [36]:
import numpy as np

def adjust_car(car):
    # Unpack car
    frame_length, upper, lower, left, right = car

    # 1. Adjust frame length
    frame_length = np.clip(frame_length, 2.0, 6.0)

    # 2. Adjust shapes
    upper = upper.copy()
    lower = lower.copy()
    for i in range(5):
        total = upper[i] + lower[i]
        if total < 0.5:
            delta = 0.5 - total
            upper[i] += delta / 2
            lower[i] += delta / 2
        elif total > 5.0:
            scale = 5.0 / total
            upper[i] *= scale
            lower[i] *= scale

    # 3. Adjust wheels
    left = left.copy()
    right = right.copy()

    left[0] = np.clip(left[0], 0.0, 1.0)
    left[1] = np.clip(left[1], 0.0, 2.0)

    right[0] = np.clip(right[0], 0.0, 1.0)
    right[1] = np.clip(right[1], 0.0, 2.0)

    # Ensure left wheel is to the left of right wheel
    # Ensure left is to the left of right with minimum spacing
    min_gap = 0.7
    if left[0] >= right[0] - min_gap:
        # Reposition wheels with gap
        mid = (left[0] + right[0]) / 2
        left[0] = np.clip(mid - min_gap / 2, 0.0, 1.0 - min_gap)
        right[0] = np.clip(left[0] + min_gap, left[0] + min_gap, 1.0)
    # # Optional: rebalance wheel sizes if needed
    # ratio = max(left[1], right[1]) / (min(left[1], right[1]) + 1e-5)
    # if ratio > 2.0:
    #     avg = (left[1] + right[1]) / 2
    #     left[1] = right[1] = np.clip(avg, 0.2, 2.0)


    # 4. Return adjusted genome
    updated_car = np.concatenate((
        [frame_length],
        upper,
        lower,
        left,
        right
    ))

    return updated_car


In [37]:
car = mate(car1, car2)
adjusted_genome = adjust_car(car)
adjusted_car = car_from_genome(adjusted_genome)

carsim.simulate(*adjusted_car)  # ✅ Now it works


### Step 8

Time for the final touches.

Write a `generation` function, to make the car population evolve by a single generation.

It takes an initial population of size P (cars) as a single argument. It should then:
1. make the cars race;
2. collect their scores;
3. select 30% of the best cars;
4. build a new car generation of size P, using:
   1. the 30% best cars selected above, say they are $N$ in number;
   1. $N$ new individuals obtained by picking pairs of best cars at random and making them mate;
   1. $P - 2N$ individuals generated totally at random (using again the function `initial_population`).

In [38]:
def generation(cars):
    P = len(cars)

    # 1. Evaluate race performance
    scores = carsim.race(cars)

    # 2. Select best 30%
    best = select_best(cars, scores, fraction=0.3)
    N = len(best)

    # 3. Mating step: generate N children by mating random best pairs
    children = []
    for _ in range(N):
        parent1 = random.choice(best)
        parent2 = random.choice(best)
        child = mate(parent1, parent2)               # crossover + mutation
        genome = adjust_car(child)                    # returns flat genome
        child_car = car_from_genome(genome)           # back to blueprint
        children.append(child_car)

    # 4. Generate (P - 2N) random cars
    random_cars = initial_population(P - 2 * N)

    # 5. New generation: best + children + randoms
    next_generation = best + children + random_cars

    return next_generation


In [39]:
import random
population = initial_population(50)

for i in range(10):  # evolve for 10 generations
    print(f"Generation {i + 1}")
    population = generation(population)

# Show final top 5
top5 = select_best(population, carsim.race(population), 5 / len(population))
for i, car in enumerate(top5):
    print(f"Top car {i+1}")
    carsim.simulate(*car)
    input("Press Enter for next car...")


Generation 1
Generation 2
Generation 3
Generation 4
Generation 5
Generation 6
Generation 7
Generation 8
Generation 9
Generation 10
Top car 1
Top car 2
Top car 3
Top car 4
Top car 5


### Step 9

Write `generations`, a functions that runs `generate` a specified number of times, returning the last available car generation.

In [40]:
def generations(cars, gen_num):
    final_cars = cars
    for i in range(gen_num):
        print(f"Generation {i + 1}")
        final_cars = generation(final_cars)
    return final_cars

### Step 10

Everything is finally set.

We make an initial population of 100 cars.

In [41]:
pop1 = initial_population(100)

We select the best individual and simulate its race to get a feeling for how well it behaves.

In [42]:
best = select_best(pop1, carsim.race(pop1), .1)[0]

In [43]:
carsim.simulate(*best)

We let evolution and selection do their work for 30 generations, and check if they actually introduced an improvement.

In [46]:
popX = generations(pop1, 30)

Generation 1
Generation 2
Generation 3
Generation 4
Generation 5
Generation 6
Generation 7
Generation 8
Generation 9
Generation 10
Generation 11
Generation 12
Generation 13
Generation 14
Generation 15
Generation 16
Generation 17
Generation 18
Generation 19
Generation 20
Generation 21
Generation 22
Generation 23
Generation 24
Generation 25
Generation 26
Generation 27
Generation 28
Generation 29
Generation 30


In [55]:
carsim.simulate(*select_best(popX, carsim.race(popX), .1)[0]);

If at this point we haven't yet reached a satisfactory design, we let the system evolve for more generations.

In [ ]:
popX = generations(popX, 30)

Generation 1
Generation 2
Generation 3
Generation 4
Generation 5
Generation 6
Generation 7
Generation 8
Generation 9
Generation 10
Generation 11
Generation 12
Generation 13
Generation 14
Generation 15
Generation 16
Generation 17
Generation 18
Generation 19
Generation 20
Generation 21
Generation 22
Generation 23
Generation 24
Generation 25
Generation 26
Generation 27
Generation 28
Generation 29
Generation 30


Notice like in the above code we evolved population X which was already the result of an evolution. If `popX` was originally 30 generation old, now it got 60 generations old.

### Step 11

When you get a working car, try changing the track profile.

In [54]:
carsim.new_track()

Is you car still fit for the new track? What does it mean if the car was good previously, but now that the track has changed is no longer working?

Lil'Swell